In [ ]:
!nvidia-smi

Fri Jun 18 06:37:12 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.27       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0    24W / 300W |      0MiB / 16160MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [1]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"mahitha1111","key":"b97b90e5726a3ba371689d223ff9c9db"}'}

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
! pip install --upgrade --force-reinstall --no-deps kaggle

     |████████████████████████████████| 61kB 6.4MB/s 
  Created wheel for kaggle: filename=kaggle-1.5.12-cp37-none-any.whl size=73053 sha256=213e6d43e4cc5bd8c43e3a3cf69b336821e4094aeca6cefa4dcc171852df830d
  Stored in directory: /root/.cache/pip/wheels/a1/6a/26/d30b7499ff85a4a4593377a87ecf55f7d08af42f0de9b60303
Successfully built kaggle
  Found existing installation: kaggle 1.5.12
    Uninstalling kaggle-1.5.12:
      Successfully uninstalled kaggle-1.5.12


In [ ]:
!mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle competitions download -c deep-learning-hw2p2-part-2-classification

 98% 583M/594M [00:13<00:00, 45.8MB/s]
100% 594M/594M [00:13<00:00, 44.7MB/s]


In [ ]:
!unzip \*.zip  && rm *.zip


Streaming output truncated to the last 5000 lines.
  inflating: val_data/val_data/2348/0001_01.jpg  
  inflating: val_data/val_data/2348/0046_01.jpg  
  inflating: val_data/val_data/2349/0082_01.jpg  
  inflating: val_data/val_data/2349/0312_02.jpg  
  inflating: val_data/val_data/235/0008_02.jpg  
  inflating: val_data/val_data/235/0140_03.jpg  
  inflating: val_data/val_data/2350/0106_01.jpg  
  inflating: val_data/val_data/2350/0339_01.jpg  
  inflating: val_data/val_data/2351/0006_01.jpg  
  inflating: val_data/val_data/2351/0150_01.jpg  
  inflating: val_data/val_data/2352/0193_01.jpg  
  inflating: val_data/val_data/2352/0311_01.jpg  
  inflating: val_data/val_data/2353/0229_01.jpg  
  inflating: val_data/val_data/2353/0297_02.jpg  
  inflating: val_data/val_data/2354/0140_01.jpg  
  inflating: val_data/val_data/2354/0444_01.jpg  
  inflating: val_data/val_data/2355/0112_02.jpg  
  inflating: val_data/val_data/2355/0385_01.jpg  
  inflating: val_data/val_data/2356/0097_01.jpg  
 

In [ ]:
#!unzip \*.zip  -d /content/drive/MyDrive/Homeworks/HW2P2

In [ ]:
import numpy as np
import torch
import PIL
import torchvision   

from torch.utils.data import Dataset, DataLoader

In [ ]:
import os
import numpy as np
from PIL import Image

import torch
import torchvision   
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt

In [ ]:
import os
from PIL import Image

In [ ]:
def parse_data(datadir):
    img_list = []
    ID_list = []
    for root, directories, filenames in os.walk(datadir):  #root: median/1
        for filename in filenames:
            if filename.endswith('.jpg'):
                filei = os.path.join(root, filename)
                img_list.append(filei)
                ID_list.append(root.split('/')[-1])
                
                # print(ID_list)

    # construct a dictionary, where key and value correspond to ID and target
    uniqueID_list = list(set(ID_list))
    class_n = len(uniqueID_list)
    target_dict = dict(zip(uniqueID_list, range(class_n)))
    target_num= dict(zip( range(class_n),uniqueID_list))
    print(target_num)
    print(target_dict)
    label_list = [target_dict[ID_key] for ID_key in ID_list]
    print(len(label_list))

    print('{}\t\t{}\n{}\t\t{}'.format('#Images', '#Labels', len(img_list), len(set(label_list))))
    return img_list, label_list, class_n,target_num


In [ ]:
img_list, label_list, class_n,target_num = parse_data('/content/train_data/train_data')
print(len(img_list),len(label_list),class_n)

{0: '2538', 1: '3164', 2: '2961', 3: '102', 4: '3797', 5: '325', 6: '3867', 7: '1793', 8: '3843', 9: '2003', 10: '3569', 11: '2181', 12: '200', 13: '282', 14: '3878', 15: '2412', 16: '3179', 17: '3695', 18: '1979', 19: '3477', 20: '2226', 21: '2147', 22: '2717', 23: '3768', 24: '2399', 25: '2002', 26: '3763', 27: '2780', 28: '3947', 29: '1616', 30: '1780', 31: '2462', 32: '1448', 33: '2035', 34: '128', 35: '1214', 36: '3437', 37: '2429', 38: '1653', 39: '3242', 40: '56', 41: '3841', 42: '759', 43: '912', 44: '1656', 45: '3556', 46: '1161', 47: '2430', 48: '1157', 49: '670', 50: '1657', 51: '871', 52: '1400', 53: '1392', 54: '1370', 55: '730', 56: '2642', 57: '757', 58: '3358', 59: '3170', 60: '3230', 61: '3029', 62: '2658', 63: '2419', 64: '1948', 65: '3105', 66: '3972', 67: '578', 68: '604', 69: '2789', 70: '701', 71: '192', 72: '1454', 73: '1551', 74: '1716', 75: '3856', 76: '257', 77: '2792', 78: '2784', 79: '1621', 80: '3575', 81: '723', 82: '1247', 83: '1542', 84: '2542', 85: '135

In [ ]:
val_img_list, val_label_list, class_n,target_num = parse_data('/content/val_data/val_data')
#a_file = open("data.pkl", "wb")
#pickle.dump(target_num, a_file)
#a_file.close()
print(len(val_img_list),len(val_label_list),class_n)

{0: '2538', 1: '3164', 2: '2961', 3: '102', 4: '3797', 5: '325', 6: '3867', 7: '1793', 8: '3843', 9: '2003', 10: '3569', 11: '2181', 12: '200', 13: '282', 14: '3878', 15: '2412', 16: '3179', 17: '3695', 18: '1979', 19: '3477', 20: '2226', 21: '2147', 22: '2717', 23: '3768', 24: '2399', 25: '2002', 26: '3763', 27: '2780', 28: '3947', 29: '1616', 30: '1780', 31: '2462', 32: '1448', 33: '2035', 34: '128', 35: '1214', 36: '3437', 37: '2429', 38: '1653', 39: '3242', 40: '56', 41: '3841', 42: '759', 43: '912', 44: '1656', 45: '3556', 46: '1161', 47: '2430', 48: '1157', 49: '670', 50: '1657', 51: '871', 52: '1400', 53: '1392', 54: '1370', 55: '730', 56: '2642', 57: '757', 58: '3358', 59: '3170', 60: '3230', 61: '3029', 62: '2658', 63: '2419', 64: '1948', 65: '3105', 66: '3972', 67: '578', 68: '604', 69: '2789', 70: '701', 71: '192', 72: '1454', 73: '1551', 74: '1716', 75: '3856', 76: '257', 77: '2792', 78: '2784', 79: '1621', 80: '3575', 81: '723', 82: '1247', 83: '1542', 84: '2542', 85: '135

In [ ]:
class ImageDataset(Dataset):
    def __init__(self, file_list, target_list,transform):
        self.file_list = file_list
        self.target_list = target_list
        self.n_class = len(list(set(target_list)))
        self.transform=transform

    def __len__(self):
        return len(self.file_list)

    def __getitem__(self, index):
        img = Image.open(self.file_list[index])
        img = self.transform(img)
        #img = torchvision.transforms.ToTensor()(img)
        label = self.target_list[index]
        return img, label


   

    

In [ ]:
from torchvision import transforms

In [ ]:

transforms = transforms.Compose([
    
    transforms.RandomHorizontalFlip(),   
    torchvision.transforms.RandomVerticalFlip(),
    transforms.RandomRotation(15),
    
    transforms.ToTensor()
    
    ])

In [ ]:
trainset = ImageDataset(img_list, label_list,transform=transforms)

In [ ]:
dataloader = DataLoader(trainset, batch_size=256, shuffle=True, num_workers=1, drop_last=False)

In [ ]:
train_data_item, train_data_label = trainset.__getitem__(73180)
print(train_data_label,train_data_item)

640 tensor([[[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]],

        [[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]],

        [[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]]])


In [ ]:
class ImageDataset(Dataset):
    def __init__(self, file_list, target_list):
        self.file_list = file_list
        self.target_list = target_list
        self.n_class = len(list(set(target_list)))
        

    def __len__(self):
        return len(self.file_list)

    def __getitem__(self, index):
        img = Image.open(self.file_list[index])
        #img = self.transform(img)
        img = torchvision.transforms.ToTensor()(img)
        label = self.target_list[index]
        return img, label


In [ ]:
valset= ImageDataset(val_img_list,val_label_list)

In [ ]:
dev_dataloader= DataLoader(valset, batch_size=256, shuffle=False, num_workers=1)

In [ ]:
val_data_item, val_data_label = trainset.__getitem__(7180)
print(val_data_label,val_data_item)

2460 tensor([[[0.0000, 0.0000, 0.3529,  ..., 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.3804,  ..., 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.4196,  ..., 0.0157, 0.0039, 0.0980],
         ...,
         [0.3529, 0.3529, 0.3529,  ..., 0.3569, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000,  ..., 0.3647, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000,  ..., 0.3686, 0.0000, 0.0000]],

        [[0.0000, 0.0000, 0.3333,  ..., 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.3608,  ..., 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.3961,  ..., 0.0118, 0.0000, 0.0941],
         ...,
         [0.3725, 0.3725, 0.3725,  ..., 0.3765, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000,  ..., 0.3843, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000,  ..., 0.3882, 0.0000, 0.0000]],

        [[0.0000, 0.0000, 0.2078,  ..., 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.2353,  ..., 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.2706,  ..., 0.0000, 0.000

In [ ]:
print('data item shape: {}\t data item label: {}'.format(train_data_item.shape, train_data_label))

data item shape: torch.Size([3, 64, 64])	 data item label: 640


In [ ]:
"""
class SimpleResidualBlock(nn.Module):
    def __init__(self, channel_size,channel_next, stride=1):
        super().__init__()
        self.conv1 = nn.Conv2d(channel_next, channel_next, kernel_size=3,padding=1, stride=stride, bias=False)
        #self.bn11 = nn.BatchNorm2d(channel_next)
        #self.conv2 = nn.Conv2d(channel_next, channel_next, kernel_size=3,padding=1, stride=stride, bias=False)
        self.bn1 = nn.BatchNorm2d(channel_next)
        self.relu1 = nn.ReLU()
        if stride == 1:
            self.shortcut = nn.Conv2d(channel_next, channel_next, kernel_size=3,padding=1, stride=stride, bias=False)
            self.bn2 = nn.BatchNorm2d(channel_next)
            #self.relu2 = nn.ReLU()
            #self.bn2 = nn.BatchNorm2d(channel_next)
        else:
            
            self.shortcut = nn.Conv2d(channel_size, channel_next, kernel_size=1, stride=2)
            self.bn2 = nn.BatchNorm2d(channel_next)
            #self.relu2 = nn.ReLU()
        #self.relu = nn.ReLU()
    
    def forward(self, x):
        out = self.conv1(x)
        out=self.bn1(out) 
        #out = self.conv2(out)
        #out = self.bn1(out)
        out=self.relu1(out)
        shortcut = self.shortcut(x)

        bbnn = self.bn2(shortcut)
        #bbnn =self.relu2(bbnn)

        
        out =out + shortcut+bbnn
        
        return out
"""

'\nclass SimpleResidualBlock(nn.Module):\n    def __init__(self, channel_size,channel_next, stride=1):\n        super().__init__()\n        self.conv1 = nn.Conv2d(channel_next, channel_next, kernel_size=3,padding=1, stride=stride, bias=False)\n        #self.bn11 = nn.BatchNorm2d(channel_next)\n        #self.conv2 = nn.Conv2d(channel_next, channel_next, kernel_size=3,padding=1, stride=stride, bias=False)\n        self.bn1 = nn.BatchNorm2d(channel_next)\n        self.relu1 = nn.ReLU()\n        if stride == 1:\n            self.shortcut = nn.Conv2d(channel_next, channel_next, kernel_size=3,padding=1, stride=stride, bias=False)\n            self.bn2 = nn.BatchNorm2d(channel_next)\n            #self.relu2 = nn.ReLU()\n            #self.bn2 = nn.BatchNorm2d(channel_next)\n        else:\n            \n            self.shortcut = nn.Conv2d(channel_size, channel_next, kernel_size=1, stride=2)\n            self.bn2 = nn.BatchNorm2d(channel_next)\n            #self.relu2 = nn.ReLU()\n        #sel

In [ ]:
class SimpleResidualBlock(nn.Module):
    def __init__(self, channel_size,channel_next, stride=1):
        super().__init__()
        self.conv1 = nn.Conv2d(channel_size, channel_next, kernel_size=3,padding=1, stride=stride, bias=False) 
            
        self.bn1 = nn.BatchNorm2d(channel_next)
        if stride == 1:
            self.shortcut = nn.Identity()
        else:
            self.shortcut = nn.Conv2d(channel_size, channel_next, kernel_size=1, stride=stride)
        self.relu = nn.ReLU()
    
    def forward(self, x):
        out = self.conv1(x)
        #out=self.conv2(x)
        out = self.bn1(out)
        
        shortcut = self.shortcut(x)
        
        out = self.relu(out + shortcut)
        
        return out

In [ ]:

class ClassificationNetwork(nn.Module):
    def __init__(self, in_features, num_classes):
        super().__init__()
        
        self.layers = nn.Sequential(
            nn.Conv2d(in_features, 64, kernel_size=3, stride=1,padding=1, bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU(),
           # SimpleResidualBlock(64,64,1), #k=4,pad=2  ==set  okkk  64
            
            SimpleResidualBlock(64,64,1),   # k=4,pad=2  =  64
            #maxpool + conv2d
            #nn.MaxPool2d(kernel_size=3, stride=2, padding=1),       # changed stride
            #SimpleResidualBlock(64,128,2),
            #nn.Conv2d(64,128,kernel_size=1, stride=1),            # changed stride
            #nn.BatchNorm2d(128),
            #SimpleResidualBlock(64,128,1), #k=4,pad=2    pad =2 32
            
            SimpleResidualBlock(128,128,1),#k=4,pad=2 65  32
            #nn.BatchNorm2d(128),
            #SimpleResidualBlock(128,256,2),
            #nn.MaxPool2d(kernel_size=3, stride=2, padding=1),     # changed stride
           
            #nn.Dropout(0.1),
            #nn.Conv2d(128,256,kernel_size=1, stride=1),
            #nn.BatchNorm2d(256),           # changed stride
           

            #SimpleResidualBlock(128,256,1),#k=4,pad=2 16
            
            #SimpleResidualBlock(256,256,2), 
            #nn.Conv2d(in_features, 64, kernel_size=3, stride=1,padding=1, bias=False),
            SimpleResidualBlock(256,256,1),
            #nn.BatchNorm2d(256),#k=4,pad=2=4,pad=2 4
            #nn.MaxPool2d(kernel_size=3, stride=1, padding=1),
            #nn.Dropout(0.1),
            #SimpleResidualBlock(256,512,2),
            #nn.MaxPool2d(kernel_size=3, stride=2, padding=1),   # changed stride
            #nn.Conv2d(256,512,kernel_size=1, stride=1),           # changed stride
            
            #nn.BatchNorm2d(512),
            SimpleResidualBlock(512,512,1),
            #nn.Dropout(0.1),
            nn.Conv2d(512,1024,kernel_size=1, stride=1),
            SimpleResidualBlock(1024,1024,1),
            #SimpleResidualBlock(512,512,2), # 2
            SimpleResidualBlock(1024,1024,1),
            nn.AdaptiveAvgPool2d((1, 1)), # For each channel, collapses (averages) the entire feature map (height & width) to 1x1
            nn.Flatten(),
            
             # the above ends up with batch_size x 512 x 1 x 1, 
        )
        
        self.linear= nn.Linear(512, 4000)
        #self.linear = nn.Linear(4500, 4000)
        # add layer

       
      
        
    def forward(self, x, return_embedding=False):
       
        embedding = self.layers(x) 
        
        if return_embedding:
            return embedding
        else:
          
          
          return self.linear(embedding)






In [ ]:
"""
class SimpleResidualBlock(nn.Module):
    def __init__(self, channel_size,channel_next, stride=1):
        super().__init__()
        self.conv1 = nn.Conv2d(channel_size, channel_next, kernel_size=3,padding=1, stride=stride, bias=False) 
        #self.conv2 = nn.Conv2d(channel_next, channel_next, kernel_size=3,padding=1, stride=1, bias=False)        
        self.bn1 = nn.BatchNorm2d(channel_next)
        if stride == 1:
            self.shortcut = nn.Identity()
        else:
            self.shortcut = nn.Conv2d(channel_size, channel_next, kernel_size=1, stride=stride)
        self.relu = nn.ReLU()
    
    def forward(self, x):
        out = self.conv1(x)
        #out=self.conv2(x)
        out = self.bn1(out)
        
        shortcut = self.shortcut(x)
        
        out = self.relu(out + shortcut)
        
        return out
"""

'\nclass SimpleResidualBlock(nn.Module):\n    def __init__(self, channel_size,channel_next, stride=1):\n        super().__init__()\n        self.conv1 = nn.Conv2d(channel_size, channel_next, kernel_size=3,padding=1, stride=stride, bias=False) \n        #self.conv2 = nn.Conv2d(channel_next, channel_next, kernel_size=3,padding=1, stride=1, bias=False)        \n        self.bn1 = nn.BatchNorm2d(channel_next)\n        if stride == 1:\n            self.shortcut = nn.Identity()\n        else:\n            self.shortcut = nn.Conv2d(channel_size, channel_next, kernel_size=1, stride=stride)\n        self.relu = nn.ReLU()\n    \n    def forward(self, x):\n        out = self.conv1(x)\n        #out=self.conv2(x)\n        out = self.bn1(out)\n        \n        shortcut = self.shortcut(x)\n        \n        out = self.relu(out + shortcut)\n        \n        return out\n'

In [ ]:
class ClassificationNetwork(nn.Module):
    def __init__(self, in_features, num_classes):
        super().__init__()        
        self.layers = nn.Sequential(
            nn.Conv2d(in_features, 64, kernel_size=3, stride=1,padding=1, bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            SimpleResidualBlock(64,64,1),
            SimpleResidualBlock(64,128,2),
            SimpleResidualBlock(128,128,1),
            SimpleResidualBlock(128,256,2),
            SimpleResidualBlock(256,256,1),
            SimpleResidualBlock(256,512,2),
            SimpleResidualBlock(512,512,1),
            nn.Conv2d(512,1024,kernel_size=1, stride=1),
            SimpleResidualBlock(1024,1024,1),
            nn.AdaptiveAvgPool2d((1, 1)), # For each channel, collapses (averages) the entire feature map (height & width) to 1x1
            nn.Flatten(),
           
        )
        nn.Dropout(0.1)
        self.linear = nn.Linear(1024, num_classes)
        
    def forward(self, x, return_embedding=False):
        embedding = self.layers(x) 
        
        if return_embedding:
            return embedding
        else:
            return self.linear(embedding)


In [ ]:
from torch.optim.lr_scheduler import StepLR

in_features = 3 # RGB channels
#learningRate = 0.01
#learningRate = 0.025
learningRate=0.1
#learningRate = 0.01
#learningRate = 1e-2
weightDecay = 5e-5
#weightDecay = 0.01
num_classes = 4000

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

network = ClassificationNetwork(in_features, num_classes)
#network=ResNet9(in_features, num_classes)
network = network.to(device)
#criterion = CenterLoss()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(network.parameters(),lr=learningRate,weight_decay=weightDecay,momentum=0.9)

#scheduler = StepLR(optimizer, step_size=1, gamma=0.1)
print(network)

ClassificationNetwork(
  (layers): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): SimpleResidualBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (shortcut): Identity()
      (relu): ReLU()
    )
    (4): SimpleResidualBlock(
      (conv1): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (shortcut): Conv2d(64, 128, kernel_size=(1, 1), stride=(2, 2))
      (relu): ReLU()
    )
    (5): SimpleResidualBlock(
      (conv1): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(128, eps=1e-05, momentum=0

In [ ]:
# Train!
#scheduler = StepLR(optimizer, step_size=10, gamma=0.4)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer,
                                                               mode='min',
                                                               factor=0.2,
                                                               patience=4,
                                                               min_lr=0.005,
                                                               verbose=True)
def train_epoch(network,dataloader,criterion,optimizer,epoch):
    # Train
    network.train()
    avg_loss = 0.0
    num_correct=0
    total_predictions=0
    accuracy=0
    num_total=0
    running_loss=0
    accur=[]

    for batch_num, (x, y) in enumerate(dataloader):
        optimizer.zero_grad()
        
        x, y = x.to(device), y.to(device)

        outputs = network(x)

        loss = criterion(outputs, y.long())
        loss.backward()
        optimizer.step()

        avg_loss += loss.item()
        value,index=torch.max(outputs,1)
        num_correct=torch.sum(index==y)
        total_predictions=outputs.shape[0]
        # print("Number of correct predictions",num_correct)
        num_total=num_total+num_correct
        batch_accuracy=num_correct/256
        # print(batch_num,"bactch accuracy is:",batch_accuracy)
        # running_loss/=len(dataloader)
        # print("Training Loss:", running_loss)


        torch.cuda.empty_cache()
        del x
        del y
        del loss
    


        # if batch_num % 10 == 9:
        #     print('Epoch: {}\tBatch: {}\tAvg-Loss: {:.4f}'.format(epoch, batch_num+1, avg_loss/))
        #     avg_loss = 0.0
    
    # # Validate
    # network.eval()
    # num_correct = 0
    # for batch_num, (x, y) in enumerate(dev_dataloader):
    #     x, y = x.to(device), y.to(device)
    #     outputs = network(x)
    #     num_correct += (torch.argmax(outputs, axis=1) == y).sum().item()
        
    # print('Epoch: {}, Validation Accuracy: {:.2f}'.format(epoch, num_correct / len(dev_dataset)))
    print("*****************************************************++++++++++++++")
    accuracy=num_total/380638
    accur.append(accuracy)
    print(epoch,"epoch: Training accuracy",accuracy)
    print("****************************************************++++++++++++++")
    path = '/content/drive/MyDrive/Homeworks/HW2P2/Models/'+str(epoch)
    torch.save(network.state_dict(), path)
    # Validate
    



In [ ]:


def val_epoch(network,dev_dataloader):
  network.eval()
  avg_loss = 0.0
  num_correct=0
  total_predictions=0
  accuracy=0
  num_total=0
  running_loss=0
  accur=[]

  for batch_num, (x, y) in enumerate(dev_dataloader):
      
      x, y = x.to(device), y.to(device)

      outputs = network(x)

    
      # print(y)

      
      value,index=torch.max(outputs,1)
      num_correct=torch.sum(index==y)
      # print(y)
      # print(index)
      total_predictions=outputs.shape[0]
      print("Number of correct predictions",num_correct)
      num_total=num_total+num_correct
      batch_accuracy=num_correct/256
      # print(batch_num,"bactch accuracy is:",batch_accuracy)
      # running_loss/=len(dataloader)
      # print("Training Loss:", running_loss)


      torch.cuda.empty_cache()
      del x
      del y



      # if batch_num % 10 == 9:
      #     print('Epoch: {}\tBatch: {}\tAvg-Loss: {:.4f}'.format(epoch, batch_num+1, avg_loss/))
      #     avg_loss = 0.0

  # # Validate
  # network.eval()
  # num_correct = 0
  # for batch_num, (x, y) in enumerate(dev_dataloader):
  #     x, y = x.to(device), y.to(device)
  #     outputs = network(x)
  #     num_correct += (torch.argmax(outputs, axis=1) == y).sum().item()
      
  # print('Epoch: {}, Validation Accuracy: {:.2f}'.format(epoch, num_correct / len(dev_dataset)))
  print("******************************************++++++++++++++")
  accuracy=num_total/8000
  print("val accuracy is:",accuracy)
  # accur.append(accuracy)
  # print(epoch,"epoch:",accuracy)
  # path = '/content/drive/MyDrive/Sujithafolder/CNN-Model'+str(epoch)
  # torch.save(network.state_dict(), path)
  # # Validate




In [ ]:
nu_epoch=400
for epoch in range(320,nu_epoch):
  scheduler.step(epoch)
  # Print Learning Rate
  #print('Epoch:', epoch,'LR:', scheduler.get_lr())
  train_epoch(network,dataloader,criterion,optimizer,epoch)
  val_epoch(network,dev_dataloader)

*****************************************************++++++++++++++
320 epoch: Training accuracy tensor(0.0072, device='cuda:0')
****************************************************++++++++++++++
Number of correct predictions tensor(7, device='cuda:0')
Number of correct predictions tensor(3, device='cuda:0')
Number of correct predictions tensor(0, device='cuda:0')
Number of correct predictions tensor(3, device='cuda:0')
Number of correct predictions tensor(2, device='cuda:0')
Number of correct predictions tensor(1, device='cuda:0')
Number of correct predictions tensor(1, device='cuda:0')
Number of correct predictions tensor(6, device='cuda:0')
Number of correct predictions tensor(3, device='cuda:0')
Number of correct predictions tensor(5, device='cuda:0')
Number of correct predictions tensor(5, device='cuda:0')
Number of correct predictions tensor(4, device='cuda:0')
Number of correct predictions tensor(3, device='cuda:0')
Number of correct predictions tensor(3, device='cuda:0')
Number

In [ ]:
# img1=Image.open('/content/val_data/val_data/3907/0360_05.jpg')
# plt.imshow(img1)


In [ ]:
# %cd /content/val_data/val_data/3907


In [ ]:
# ls

In [ ]:
# path='/content/drive/MyDrive/Sujithafolder/CNN-Model10'
# state_dict = torch.load(path)
# #print(len(state_dict["net.14.weight"]))
# network.load_state_dict(state_dict)
# # print(state_dict)
# # print(network)

# network.eval()
# val_num_correct = 0
# total_num_correct=0
# total_predictions=0
# bactchaccuracy=0
# for batch_num, (x, y) in enumerate(dev_dataloader):
#     x, y = x.to(device), y.to(device)
#     # print(x ,"===========================")
#     val_outputs = network(x)
#     value,index=torch.max(val_outputs,1)
#     val_num_correct=torch.sum(index==y)
#     print("correct predictions:",val_num_correct)
#     total_num_correct+=val_num_correct
#     total_predictions=val_outputs.shape[0]
#     bactchaccuracy=val_num_correct/total_predictions
#     print(bactchaccuracy)
# accuracy=total_num_correct/8000
# print("**************************",accuracy,"****************************")


In [ ]:
#  loading the model.

# path='/content/drive/MyDrive/Sujithafolder/CNN-Model10'
# state_dict = torch.load(path)
# #print(len(state_dict["net.14.weight"]))
# network.load_state_dict(state_dict)

In [ ]:
# val_epoch(network,dev_dataloader)

In [ ]:
state_dict = torch.load(path,map_location=torch.device('cpu'))
# #print(len(state_dict["net.14.weight"]))
network.load_state_dict(state_dict)


In [ ]:

def parse_data(datadir):
    img_list = []
    ID_list = []
    for root, directories, filenames in os.walk(datadir):  #root: median/1
        for filename in filenames:
            if filename.endswith('.jpg'):
                filei = os.path.join(root, filename)
                img_list.append(filei)
                ID_list.append(root.split('/')[-1])
                # print(ID_list)

    # construct a dictionary, where key and value correspond to ID and target
    # uniqueID_list = list(set(ID_list))
    # class_n = len(uniqueID_list)
    # target_dict = dict(zip(uniqueID_list, range(class_n)))
    # print(target_dict)
    # label_list = [target_dict[ID_key] for ID_key in ID_list]
    # print(len(label_list))
    class_n=4000

    # print('{}\t\t{}\n{}\t\t{}'.format('#Images', '#Labels', len(img_list), len(set(label_list))))
    return img_list, class_n


In [ ]:
# from PIL import Image


# img=Image.open('/content/val_data/val_data/3120/0001_01.jpg')
# plt.imshow(img)


class ImageDatatest(Dataset):
    def __init__(self, file_list):
        self.file_list = file_list
        # self.target_list = target_list
        # self.n_class = len(list(set(target_list)))
        self.n_class=4000

    def __len__(self):
        return len(self.file_list)

    def __getitem__(self, index):
        img = Image.open(self.file_list[index])
        img = torchvision.transforms.ToTensor()(img)
        # label = self.target_list[index]
        return img



In [ ]:
test_img_list, class_n = parse_data('/content/test_data/test_data')
testset = ImageDatatest(test_img_list)
test_dataloader= DataLoader(testset, batch_size=256, shuffle=False, num_workers=1)
test_data_item= testset.__getitem__(23)
print(target_num)

In [ ]:
def test_epoch(network,test_dataloder):
  
  network.eval()
  avg_loss = 0.0
  num_correct=0
  total_predictions=0
  accuracy=0
  num_total=0
  running_loss=0
  accur=[]
  result=[]


  for batch_idx, data in enumerate(test_dataloader):

    data=data.to(device)
    outputs=network(data) 
    # print(outputs.shape)
    value,index=torch.max(outputs,1)
    result.append(index.tolist())
  return result 
      

      
      





In [ ]:
result=test_epoch(network,test_dataloader)

In [ ]:
images=[]
for i in range(len(test_img_list)):
  images.append(test_img_list[i].split("/")[-1])
print(images)

In [ ]:
tensor_result=torch.as_tensor(result[0:-1]).flatten()
tensor_result.shape
last=torch.as_tensor(result[-1])
final_tensor=torch.hstack((tensor_result,last))
final=final_tensor.tolist()
from pandas import DataFrame
# your_list = ['item1', 'item2', 'item3',...]
# df = DataFrame (images,final,columns=['id','labels'])
df = DataFrame(final, index =images,columns =['label'])
df.to_csv("sample.csv",index_label='id')
df.head


In [ ]:
# import the csv file
import pandas as pd
file=pd.read_csv('/content/sample.csv')
# print(file.head)
li=[]
news=[]
# print("The images are",file['id'])
li=file['label']
# print(target_num)
for i in li:
  news.append((target_num[i]))
# print(news)
file['label']=news
file.to_csv('sample.csv')
file.head(5)



In [ ]:
#!kaggle competitions submit -c deep-learning-hw2p2-part-2-classification -f heroine.csv -m "first"